# Beautiful Plots of Clifford Attractors

Requirements:
- NumPy
- Cython
- Matplotlib

In [ ]:
%load_ext cython
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

## Main function

The following Cython code does all the heavy lifting:

In [ ]:
%%cython -c=-O3

import cython

import numpy as np
cimport numpy as np

from libc.math cimport cos, sin


@cython.boundscheck(False)
def clifford_trajectory(int n, double a, double b, double c, double d):
    """Compute Clifford Attractor as a parametric curve"""
    cdef np.ndarray[ndim=1, dtype=np.float_t] x, y
    cdef int i
    
    x, y = np.empty(n), np.empty(n)
    x[0] = y[0] = 0
    for i in range(1, n):
        x[i] = sin(a * y[i-1]) + c * cos(a * x[i-1])
        y[i] = sin(b * x[i-1]) + d * cos(b * y[i-1])
    return x, y


@cython.cdivision(True)
@cython.boundscheck(False)
def clifford_bincount(int n, int bins_x, int bins_y, double a, double b, double c, double d):
    """Compute Clifford Attractor as a 2D histogram
    
    Arguments:
        n (int): Number of iterations to calculate (the more the denser)
        bins_x, bins_y (int): Number of bins per dimension (dimensions
            of the resulting histogram)
        a, b, c, d (float): Clifford Attractor parameters
        
    """
    cdef np.ndarray[ndim=2, dtype=np.float32_t] bincount
    cdef double x, x_prev, y, y_prev
    cdef int i, idx, idy
    
    cdef double min_x, max_x, min_y, max_y
    min_x = -1 - abs(c)
    max_x = 1 + abs(c)
    min_y = -1 - abs(d)
    max_y = 1 + abs(d)
    
    bincount = np.zeros((bins_x, bins_y), dtype=np.float32)
    x_prev = y_prev = 0
    
    for i in range(1, n):
        x = sin(a * y_prev) + c * cos(a * x_prev)
        y = sin(b * x_prev) + d * cos(b * y_prev)
        idx = int(bins_x * (x - min_x) / (max_x - min_x))
        idy = int(bins_y * (y - min_y) / (max_y - min_y))
        
        bincount[idx, idy] += 1
        # also increment adjacent pixels to get a nice, smooth brush stroke
        if idx > 0:
            bincount[idx-1, idy] += 0.25
        if idy > 0:
            bincount[idx, idy-1] += 0.25
        if idx < bins_x-1:
            bincount[idx+1, idy] += 0.25
        if idy < bins_y-1:
            bincount[idx, idy+1] += 0.25
            
        x_prev, y_prev = x, y
    return bincount

## Let's compute some random attractors!

Repeat computation for boring parameter combinations (short limit cycles)

In [ ]:
def normalize(clifford):
    """Increase contrast by plotting logarithm of histogram count"""
    return np.log10(1 + clifford)

In [ ]:
i = 0
while i < 20:
    a, b, c, d = map(float, 4 * np.round(np.random.rand(4,1), 2) - 2)
    cliff = clifford_bincount(10**7, 1024, 1024, a, b, c, d)
    
    if np.count_nonzero(cliff) < cliff.size / 10:
        continue
        
    fig = plt.figure(figsize=(5, 5), dpi=100)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    fig.suptitle('a: %s; b: %s; c: %s; d: %s' % (a, b, c, d), color='0.1', y=1.05)
    ax.imshow(normalize(cliff), interpolation='bilinear', cmap='magma')
    
    i += 1

## Now, let's find the best color map 🌈

and make it pretty!

In [ ]:
cmaps = ('viridis', 'plasma', 'inferno', 'magma',
         'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
         'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
         'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
         'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
         'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
         'hot', 'afmhot', 'gist_heat', 'copper')

In [ ]:
a, b, c, d = -1.52, 1.64, 1.08, 1.88

# hi-res, baby!
cliff = clifford_bincount(10**8, 2048, 2048, a, b, c, d)

In [ ]:
for cmap in cmaps:
    fig = plt.figure(figsize=(5, 5), dpi=100)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    fig.suptitle(cmap, color='0.1', y=1.05)
    ax.imshow(np.fliplr(normalize(cliff)), interpolation='bilinear', cmap=cmap, aspect='equal')
    fig.savefig('%s.png' % cmap, dpi=300)

### ... aaaand we're done!

Enjoy :)